In [1]:
import polars as pl

In [2]:
df = pl.read_parquet('../../Статья 3/Article 3 Data/posts/preprocessed.parquet')

In [3]:
df

author,id,timestamp,message,tickers,prices,n_comments,reactions,n_reactions,date_1,date_2,date_3,date_4,date_5,comment_1,comment_2,comment_3,comment_4,comment_5
str,str,datetime[μs],str,list[str],list[str],i64,list[str],i64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],str,str,str,str,str
"""T-Technology""","""87e771e7-21a0-40d9-89f5-332fb6…",2017-11-15 10:45:00,"""ТКС Банк завершил сделку по по…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""T-Technology""","""45aff112-4a6a-4014-9f04-6d5918…",2017-11-15 10:45:00,"""Евгений Бернштам в программе &…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""T-Technology""","""913bd4d9-d158-4ecb-b66b-180cc9…",2017-11-15 10:45:00,"""ТКС Банк начал сотрудничать со…","[""MTSS"", ""EURRUB""]","[""279,8₽−26,77%"", ""71,01₽+34,67%""]",null,null,null,null,null,null,null,null,null,null,null,null,null
"""T-Technology""","""73b6b21b-33d8-4a97-9a55-cefa09…",2017-11-15 10:45:00,"""ТКС Банк первым из российских …",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""T-Technology""","""0c16af14-10e0-4eb0-ad59-5a6005…",2017-11-15 10:45:00,"""ТКС Банк запустил «Тинькофф Мо…","[""MSFT"", ""MA""]","[""82,98$+411,69%"", ""148,33$+251,91%""]",null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""vasy4567""","""db853be7-7275-4803-b6db-7cfaea…",2025-04-13 17:20:00,"""$PHORкак эти тролли и зловонны…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Sukin_sin""","""58ea8519-e34a-49e4-a01f-fa6203…",2025-04-13 17:20:00,"""$UNAC рожденный летать утонуть…","[""UNAC""]","[""0,59₽−0,17%""]",null,null,null,null,null,null,null,null,null,null,null,null,null
"""DmGWT""","""f8168433-da00-4a2f-b98e-27790f…",2025-04-13 17:21:00,"""$EUTR Медленно, но верно плано…","[""EUTR""]","[""126,2₽−2,14%""]",null,"[""083b50db-7d1b-43a0-aae3-88a73e8c3c39?size=small""]",9,null,null,null,null,null,null,null,null,null,null
